In [10]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

In [2]:
# Load data
training_data = pd.read_csv("train_essays.csv")
test_data = pd.read_csv("test_essays.csv")

In [3]:
# Custom preprocessing function
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Tokenization, stopwords removal, and other steps are already covered by TfidfVectorizer
    
    return text

In [4]:
# Split the data
X_train,  X_validation, y_train,y_validation = train_test_split(training_data['text'], training_data['generated'], test_size=0.2, random_state=42)


In [5]:
# Create a pipeline with TF-IDF vectorizer and Support Vector Machine classifier
model = make_pipeline(
    TfidfVectorizer(preprocessor=preprocess_text),
    SVC(probability=True, random_state=42)
)

In [6]:
# Hyperparameter tuning using GridSearchCV
param_grid = {
    'svc__C': [0.1, 1, 10],
    'svc__kernel': ['linear', 'rbf'],
    'tfidfvectorizer__max_features': [5000, 10000, None],
}

In [7]:
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
# Best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

C:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Best Hyperparameters: {'svc__C': 0.1, 'svc__kernel': 'linear', 'tfidfvectorizer__max_features': 5000}


In [8]:
# Validate the model
y_validation_pred = grid_search.predict(X_validation)
print(classification_report(y_validation, y_validation_pred))

# Print accuracy
accuracy = grid_search.score(X_validation, y_validation)
print("Accuracy:", accuracy)

C:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       275
           1       0.00      0.00      0.00         1

    accuracy                           1.00       276
   macro avg       0.50      0.50      0.50       276
weighted avg       0.99      1.00      0.99       276

Accuracy: 0.9963768115942029


In [9]:
# Make predictions on the test set
test_predictions = grid_search.predict_proba(test_data['text'])[:, 1]

# Create a submission file
submit = pd.DataFrame({'id': test_data['id'], 'generated': test_predictions})
submit.to_csv("submission.csv", index=False)